<a href="https://colab.research.google.com/github/NitinGautam05/Plant-leaf-disease-identification/blob/main/Conv5_%26_inception_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import zipfile
import io

# URL of the ZIP file
url = "https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the ZIP file
    with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:
        # Extract all the contents to a directory
        zip_ref.extractall("")
    print("Extraction successful.")
else:
    print("Failed to download the ZIP file.")

Extraction successful.


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def inception_v3(input_shape, num_classes):
    # Define the input tensor
    inputs = tf.keras.Input(shape=input_shape)

    # Stem
    x = layers.Conv2D(32, (3, 3), strides=(2, 2), activation='relu')(inputs)
    x = layers.Conv2D(32, (3, 3), activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = layers.Conv2D(80, (1, 1), activation='relu')(x)
    x = layers.Conv2D(192, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    # Inception blocks
    x = inception_block(x, filters=[64, 96, 128, 16, 32, 32])
    x = inception_block(x, filters=[128, 128, 192, 32, 96, 64])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = inception_block(x, filters=[192, 96, 208, 16, 48, 64])
    x = inception_block(x, filters=[160, 112, 224, 24, 64, 64])
    x = inception_block(x, filters=[128, 128, 256, 24, 64, 64])
    x = inception_block(x, filters=[112, 144, 288, 32, 64, 64])
    x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
    x = layers.MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = inception_block(x, filters=[256, 160, 320, 32, 128, 128])
    x = inception_block(x, filters=[384, 192, 384, 48, 128, 128])

    # Final layers
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    # Create model
    model = models.Model(inputs, outputs, name='inception_v3')
    return model

def inception_block(x, filters):
    # Filter sizes
    filters1, filters3_reduce, filters3, filters5_reduce, filters5, filters_pool = filters

    # 1x1 conv
    path1 = layers.Conv2D(filters1, (1, 1), activation='relu')(x)

    # 1x1 conv -> 3x3 conv
    path2 = layers.Conv2D(filters3_reduce, (1, 1), activation='relu')(x)
    path2 = layers.Conv2D(filters3, (3, 3), activation='relu', padding='same')(path2)

    # 1x1 conv -> 5x5 conv
    path3 = layers.Conv2D(filters5_reduce, (1, 1), activation='relu')(x)
    path3 = layers.Conv2D(filters5, (5, 5), activation='relu', padding='same')(path3)

    # 3x3 pool -> 1x1 conv
    path4 = layers.MaxPooling2D((3, 3), strides=(1, 1), padding='same')(x)
    path4 = layers.Conv2D(filters_pool, (1, 1), activation='relu')(path4)

    # Concatenate paths
    return tf.keras.layers.concatenate([path1, path2, path3, path4], axis=-1)

# Define dataset directory
data_dir = 'Plant_leave_diseases_dataset_without_augmentation'

# Define image dimensions and batch size
img_width, img_height = 299, 299
batch_size = 32

# Use ImageDataGenerator to load and preprocess images
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Create Inception-v3 model
num_classes = len(train_generator.class_indices)
inception_model = inception_v3((img_width, img_height, 3), num_classes)

# Compile the model
inception_model.compile(optimizer='adam',
                        loss='categorical_crossentropy',
                        metrics=['accuracy'])

inception_model.summary()

Train the model
history = inception_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)




Found 44371 images belonging to 39 classes.
Found 11077 images belonging to 39 classes.
Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         896       ['input_1[0][0]']             
                                                                                                  
 conv2d_1 (Conv2D)           (None, 147, 147, 32)         9248      ['conv2d[0][0]']              
                                                                                                  
 conv2d_2 (Conv2D)           (None, 147, 147, 64)         18496     ['conv2d_1[0][0]']            

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Evaluate the model on the test set
test_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False  # Ensure that the order is preserved
)

test_steps = np.ceil(test_generator.samples / batch_size)
test_loss, test_accuracy = inception_model.evaluate(test_generator, steps=test_steps)

# Generate predictions for the test set
test_generator.reset()  # Reset the generator to start from the beginning
y_pred = inception_model.predict(test_generator, steps=test_steps)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator.classes

# Calculate precision, recall, and F1 score
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print test accuracy, precision, recall, and F1 score
print("Test Accuracy:", test_accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
# Conv5
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img
from tensorflow.keras import layers, models

data_dir = 'Plant_leave_diseases_dataset_without_augmentation'

def resize_image(img):
    img = img_to_array(img)
    img = tf.image.resize(img, (128, 128))
    return img.numpy()

batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=resize_image
)


train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Check the target image size
img_height, img_width, img_channels = train_generator.image_shape
print("img_width =", img_width)
print("img_height =", img_height)


Found 44371 images belonging to 39 classes.
Found 11077 images belonging to 39 classes.
img_width = 128
img_height = 128


In [ ]:

# Define the Conv5 model
model = models.Sequential([
    layers.Conv2D(4, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(16, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(2048, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 4)       112       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 4)         0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 16)        592       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 16, 16, 32)        0

In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)

Epoch 1/10
1386/1386 [==============================] - 355s 251ms/step - loss: 1.8970 - accuracy: 0.4622 - val_loss: 1.2198 - val_accuracy: 0.6311
Epoch 2/10
1386/1386 [==============================] - 345s 249ms/step - loss: 0.8944 - accuracy: 0.7219 - val_loss: 0.6444 - val_accuracy: 0.8018
Epoch 3/10
1386/1386 [==============================] - 342s 247ms/step - loss: 0.6151 - accuracy: 0.8028 - val_loss: 0.7264 - val_accuracy: 0.7701
Epoch 4/10
1386/1386 [==============================] - 358s 258ms/step - loss: 0.4898 - accuracy: 0.8384 - val_loss: 0.5170 - val_accuracy: 0.8396
Epoch 5/10
1386/1386 [==============================] - 343s 248ms/step - loss: 0.3996 - accuracy: 0.8690 - val_loss: 0.3882 - val_accuracy: 0.8769
Epoch 6/10
1386/1386 [==============================] - 342s 247ms/step - loss: 0.3555 - accuracy: 0.8819 - val_loss: 0.3181 - val_accuracy: 0.9006
Epoch 7/10
1386/1386 [==============================] - 362s 261ms/step - loss: 0.2989 - accuracy: 0.9013 - val_